````sh
 pip install requests ipykernel Pillow     
````

In [ ]:
# CONFIG

__longitude__ = 9.37477
__latitude__ = 47.42391
__plant__ = 'Wheat'

import risks
import img_generation
import supporting

In [ ]:
# Calculate stresses and risks
risks_bool, _ = supporting.create_risk_report({'longitude': __longitude__, 'latitude': __latitude__, 'plant': __plant__})
print(risks_bool)

In [ ]:
# risks_bool={'diurnal_stress': 1, 'nighttime_stress': 1, 'frost_stress': 1, 'drought_risk': 1, 'yield_risk': 1} # test

elements = [
    {
        "header": "Elevated Temperature Alert!",
        "body": "Hey there! It looks like the maximum temperature is set to rise. To keep your crops happy and healthy, I highly recommend applying Stress Buster. It’s your best friend in tackling heat stress and ensuring your plants thrive!",
        "type": "warning",
        "icon": "Icons/temperature-arrow-up-solid.png",
        "status": (risks_bool['diurnal_stress'] | risks_bool["nighttime_stress"])
    },
    {
        "header": "Low Moisture Warning!",
        "body": "Heads up! A drought is on the horizon. Don’t let your crops suffer—apply Stress Buster to keep them hydrated and flourishing. Your plants will thank you for it!",
        "type": "warning",
        "icon": "Icons/sun-plant-wilt-solid.png",
        "status": risks_bool["drought_risk"]
    },
    {
        "header": "Frost Warning!",
        "body": "Brrr! Frost is expected in the coming days. Protect your precious crops by applying Stress Buster. It’s the perfect shield to ensure they stay safe and sound during chilly nights!",
        "type": "warning",
        "icon": "Icons/snowflake-regular.png",
        "status": risks_bool["frost_stress"]
    },
    {
        "header": "Yield Risk Alert!",
        "body": "Uh-oh! Based on the latest data, your yield might be at risk of falling short. Don’t worry—apply Yield Booster to give your crops the extra boost they need for a bountiful harvest. Let’s make sure you get the best out of your fields!",
        "type": "warning",
        "icon": "Icons/arrow-down-wide-short-solid.png",
        "status": risks_bool["yield_risk"]
    }
]
city_name = risks.get_city_name(__longitude__, __latitude__)

img = img_generation.generate(elements, __plant__, city_name)
img.show()